# TFM. Modelo predictivo llamadas Call Center
## MVP

## Importacion de librerias 

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import datetime
import calendar

pd.options.display.max_columns = None

### Importacion de los datos de envios por oficinas

In [2]:
df_shippings = pd.read_csv('data/shippings.csv', low_memory=False)

In [3]:
df_shippings['Franquicia destino'].unique()

array([4800, 4802, 4803, 4806, 4810])

##### Filtramos oficinas

In [4]:
#df = df_shippings[ (df_shippings['Franquicia destino']==4800) ]

In [5]:
df = df_shippings
df.columns

Index(['Unnamed: 0', 'Fecha envío', 'Número envío', 'Id. Fiscal',
       'Nombre Comercial', 'Código servicio', 'Nombre Rem', 'Población Rem',
       'CP Rem', 'Nombre vía Rem', 'Nombre', 'Población', 'Código postal',
       'Nombre vía', 'Total bultos', 'Franquicia origen', 'Franquicia destino',
       'Total', 'Estado', 'Tipo anomalía', 'Motivo', 'Importe Total'],
      dtype='object')

In [6]:
df = df.iloc[:, [1,5,12,14,15,16]]
df.head()

,Fecha envío,Código servicio,Código postal,Total bultos,Franquicia origen,Franquicia destino
0,01/01/2020,Urgente 12,07011,1,636,4800
1,01/01/2020,Económico,07003,2,640,4800
2,01/01/2020,Económico,07001,1,642,4800
3,01/01/2020,Urgente 10,07012,1,679,4800
4,01/01/2020,E-commerce,07003,1,2638,4800


In [7]:
df['Fecha envío'] = pd.to_datetime(df['Fecha envío'])

<ipython-input-7-90ac39e29746>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fecha envío'] = pd.to_datetime(df['Fecha envío'])


#### Filtramos fechas en el dataset

In [8]:
#df = df[df['Fecha envío'] > '01-6-2020']
#df.head()

In [9]:
def updateDate(s):
    envio24h = ['Urgente 8:30', 'Urgente 10','Urgente 12','Urgente 14','Urgente 19', \
               'Urgente 8:30 expedición', 'Urgente 10 expedición','Urgente 12 expedición', \
               'Urgente 14 expedición','Urgente 19 expedición','Urgente hoy','Burofax', \
               'Documentos 14','Mascotas','Bag 14','Bag 19','Express documentos']
    envio48h = ['Económico','E-commerce','Mrw economy (ecopaq)','Retorno canje']
    maritimo = ['Marítimo baleares']
    
    nDate = np.nan
    ## Arreglar los 24H
    if s['Código servicio'] in envio24h:
        nDate = s['Fecha envío'] + datetime.timedelta(days=2)
    elif s['Código servicio'] in envio48h:
        nDate = s['Fecha envío'] + datetime.timedelta(days=3)
    else:
        nDate = s['Fecha envío'] + datetime.timedelta(days=4)
    
    if nDate.isoweekday() == 7:
        nDate = nDate + datetime.timedelta(days=1)
    elif nDate.isoweekday() == 6:
        nDate = nDate + datetime.timedelta(days=2)
        
    return nDate

In [10]:
df['xDate'] = df.apply(updateDate, axis=1)

<ipython-input-10-99700ddc10e6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['xDate'] = df.apply(updateDate, axis=1)


In [11]:
df[df['xDate'].isnull()]

,Fecha envío,Código servicio,Código postal,Total bultos,Franquicia origen,Franquicia destino,xDate


In [12]:
df['Fecha envío'] =  pd.to_datetime(df['xDate'])

<ipython-input-12-9e72888f5bd0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fecha envío'] =  pd.to_datetime(df['xDate'])


In [13]:
dias = list(calendar.day_name)

In [14]:
df['dayofweek'] = [dias[numero] for numero in df['Fecha envío'].dt.dayofweek]
df.head()

<ipython-input-14-0dfaeb8ecb17>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayofweek'] = [dias[numero] for numero in df['Fecha envío'].dt.dayofweek]


,Fecha envío,Código servicio,Código postal,Total bultos,Franquicia origen,Franquicia destino,xDate,dayofweek
0,2020-01-03,Urgente 12,07011,1,636,4800,2020-01-03,Friday
1,2020-01-06,Económico,07003,2,640,4800,2020-01-06,Monday
2,2020-01-06,Económico,07001,1,642,4800,2020-01-06,Monday
3,2020-01-03,Urgente 10,07012,1,679,4800,2020-01-03,Friday
4,2020-01-06,E-commerce,07003,1,2638,4800,2020-01-06,Monday


In [15]:
df['Fecha envío'] =  pd.to_datetime(df['Fecha envío']).astype('str')

<ipython-input-15-bee2ca5c2dd9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fecha envío'] =  pd.to_datetime(df['Fecha envío']).astype('str')


In [16]:
df['Código servicio'].unique()

array(['Urgente 12', 'Económico', 'Urgente 10', 'E-commerce',
       'Urgente 19', 'Mascotas', 'Marítimo baleares', 'Urgente 14',
       'Urgente 19 expedición', 'Urgente 12 expedición', 'Urgente 8:30',
       'Burofax', 'Retorno albarán', 'Bag 14', 'Urgente 10 expedición',
       'Urgente hoy', 'Documentos 14', 'Bag 19',
       'Urgente 8:30 expedición', 'Mrw economy (ecopaq)',
       'Urgente 14 expedición', 'Express documentos', 'Retorno canje',
       'Urgente funchal', 'Express paquetes', 'Devolución burofax',
       'E-Commerce', 'Plan ayuda', 'Ecobox 25',
       'OperativaServicioEnumECOBA', 'E-commerce canje',
       'Retorno burofax'], dtype=object)

In [17]:
train = df[['Fecha envío','Código servicio','Franquicia origen','Código postal','dayofweek']]
train.reset_index(inplace=True)
train = train[['Fecha envío','Código servicio','Franquicia origen','Código postal','dayofweek']]
train['total'] = 1

In [18]:
train.head()

,Fecha envío,Código servicio,Franquicia origen,Código postal,dayofweek,total
0,2020-01-03,Urgente 12,636,07011,Friday,1
1,2020-01-06,Económico,640,07003,Monday,1
2,2020-01-06,Económico,642,07001,Monday,1
3,2020-01-03,Urgente 10,679,07012,Friday,1
4,2020-01-06,E-commerce,2638,07003,Monday,1


In [19]:
train['Franquicia origen'] = train['Franquicia origen'].apply(str)
train['Código postal'] = train['Código postal'].apply(str)

In [20]:
encoder = OneHotEncoder(handle_unknown='ignore')
enc = train[['Código servicio']]

In [21]:
encoder.fit(enc.values.reshape(-1,1))
encoder.categories_

[array(['Bag 14', 'Bag 19', 'Burofax', 'Devolución burofax',
        'Documentos 14', 'E-Commerce', 'E-commerce', 'E-commerce canje',
        'Ecobox 25', 'Económico', 'Express documentos', 'Express paquetes',
        'Marítimo baleares', 'Mascotas', 'Mrw economy (ecopaq)',
        'OperativaServicioEnumECOBA', 'Plan ayuda', 'Retorno albarán',
        'Retorno burofax', 'Retorno canje', 'Urgente 10',
        'Urgente 10 expedición', 'Urgente 12', 'Urgente 12 expedición',
        'Urgente 14', 'Urgente 14 expedición', 'Urgente 19',
        'Urgente 19 expedición', 'Urgente 8:30', 'Urgente 8:30 expedición',
        'Urgente funchal', 'Urgente hoy'], dtype=object)]

In [22]:
encoded = encoder.transform(enc.values.reshape(-1,1))
encoded_df = pd.DataFrame(encoded.todense())
encoded_df.columns = encoder.categories_[0]

In [23]:
train = pd.concat([train,encoded_df], axis=1)

In [24]:
#encoder = OneHotEncoder(handle_unknown='ignore')
enc = train[['dayofweek']]
encoder.fit(enc.values.reshape(-1,1))
encoded = encoder.transform(enc.values.reshape(-1,1))
encoded_df = pd.DataFrame(encoded.todense())
encoded_df.columns = encoder.categories_[0]

In [25]:
train = pd.concat([train,encoded_df], axis=1)

In [26]:
#encoder = OneHotEncoder(handle_unknown='ignore')
#enc = train[['Código postal']]
#encoder.fit(enc.values.reshape(-1,1))
#encoded = encoder.transform(enc.values.reshape(-1,1))
#encoded_df = pd.DataFrame(encoded.todense())
#encoded_df.columns = encoder.categories_[0]

In [27]:
#train = pd.concat([train,encoded_df], axis=1)

In [28]:
train.drop(columns=['Código servicio','Franquicia origen','Código postal','dayofweek'], inplace=True)

In [29]:
train = train.groupby(['Fecha envío']).sum()

In [30]:
train.head()

,total,Bag 14,Bag 19,Burofax,Devolución burofax,Documentos 14,E-Commerce,E-commerce,E-commerce canje,Ecobox 25,Económico,Express documentos,Express paquetes,Marítimo baleares,Mascotas,Mrw economy (ecopaq),OperativaServicioEnumECOBA,Plan ayuda,Retorno albarán,Retorno burofax,Retorno canje,Urgente 10,Urgente 10 expedición,Urgente 12,Urgente 12 expedición,Urgente 14,Urgente 14 expedición,Urgente 19,Urgente 19 expedición,Urgente 8:30,Urgente 8:30 expedición,Urgente funchal,Urgente hoy,Friday,Monday,Thursday,Tuesday,Wednesday
Fecha envío,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-03,30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,21.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0
2020-01-06,1126,0.0,1.0,1.0,0.0,0.0,0.0,161.0,0.0,0.0,35.0,0.0,0.0,152.0,6.0,0.0,0.0,0.0,1.0,0.0,0.0,69.0,1.0,164.0,16.0,0.0,0.0,497.0,22.0,0.0,0.0,0.0,0.0,0.0,1126.0,0.0,0.0,0.0
2020-01-07,734,0.0,0.0,0.0,0.0,0.0,0.0,396.0,0.0,0.0,142.0,0.0,0.0,186.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,734.0,0.0
2020-01-08,1105,0.0,0.0,6.0,0.0,0.0,0.0,23.0,0.0,0.0,4.0,0.0,0.0,595.0,23.0,0.0,0.0,0.0,1.0,0.0,0.0,127.0,6.0,172.0,12.0,1.0,0.0,100.0,33.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1105.0
2020-01-09,2414,0.0,0.0,8.0,0.0,0.0,0.0,1381.0,0.0,0.0,382.0,0.0,0.0,97.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,129.0,6.0,176.0,14.0,0.0,0.0,144.0,60.0,2.0,1.0,0.0,0.0,0.0,0.0,2414.0,0.0,0.0


In [31]:
train.columns

Index(['total', 'Bag 14', 'Bag 19', 'Burofax', 'Devolución burofax',
       'Documentos 14', 'E-Commerce', 'E-commerce', 'E-commerce canje',
       'Ecobox 25', 'Económico', 'Express documentos', 'Express paquetes',
       'Marítimo baleares', 'Mascotas', 'Mrw economy (ecopaq)',
       'OperativaServicioEnumECOBA', 'Plan ayuda', 'Retorno albarán',
       'Retorno burofax', 'Retorno canje', 'Urgente 10',
       'Urgente 10 expedición', 'Urgente 12', 'Urgente 12 expedición',
       'Urgente 14', 'Urgente 14 expedición', 'Urgente 19',
       'Urgente 19 expedición', 'Urgente 8:30', 'Urgente 8:30 expedición',
       'Urgente funchal', 'Urgente hoy', 'Friday', 'Monday', 'Thursday',
       'Tuesday', 'Wednesday'],
      dtype='object')

In [32]:
train.describe()

,total,Bag 14,Bag 19,Burofax,Devolución burofax,Documentos 14,E-Commerce,E-commerce,E-commerce canje,Ecobox 25,Económico,Express documentos,Express paquetes,Marítimo baleares,Mascotas,Mrw economy (ecopaq),OperativaServicioEnumECOBA,Plan ayuda,Retorno albarán,Retorno burofax,Retorno canje,Urgente 10,Urgente 10 expedición,Urgente 12,Urgente 12 expedición,Urgente 14,Urgente 14 expedición,Urgente 19,Urgente 19 expedición,Urgente 8:30,Urgente 8:30 expedición,Urgente funchal,Urgente hoy,Friday,Monday,Thursday,Tuesday,Wednesday
count,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000
mean,2595.009934,0.619205,0.288079,5.089404,0.016556,0.549669,0.009934,657.963576,0.009934,0.003311,263.715232,0.026490,0.029801,1241.824503,15.198675,0.387417,0.003311,0.003311,3.298013,0.003311,0.006623,102.453642,4.278146,152.092715,22.980132,1.294702,0.029801,74.089404,45.632450,2.029801,1.062914,0.006623,0.013245,467.364238,1243.145695,386.688742,270.142384,227.668874
std,2232.415118,2.535599,0.615247,7.464776,0.207158,0.879359,0.099337,670.898487,0.099337,0.057544,285.825037,0.160854,0.188822,1273.280313,14.167493,1.074736,0.057544,0.057544,4.960702,0.057544,0.081244,80.933566,4.358760,120.072921,21.123736,2.222485,0.274811,66.156479,42.789922,2.459460,1.037190,0.081244,0.114512,1052.766291,2666.962766,864.174648,634.291210,570.127726
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1060.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,29.000000,0.000000,0.000000,4.000000,0.000000,0.000000,216.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.250000,0.000000,4.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1919.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,525.000000,0.000000,0.000000,256.500000,0.000000,0.000000,934.000000,15.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,111.500000,3.000000,164.000000,20.000000,0.000000,0.000000,73.500000,46.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3125.750000,0.000000,0.000000,7.000000,0.000000,1.000000,0.000000,998.750000,0.000000,0.000000,354.500000,0.000000,0.000000,1845.500000,24.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,138.000000,7.000000,200.000000,34.750000,2.000000,0.000000,98.750000,61.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,11342.000000,25.000000,4.000000,67.000000,3.000000,4.000000,1.000000,3005.000000,1.000000,1.000000,1466.000000,1.000000,2.000000,7045.000000,72.000000,11.000000,1.000000,1.000000,31.000000,1.000000,1.000000,439.000000,19.000000,552.000000,121.000000,16.000000,4.000000,497.000000,443.000000,13.000000,5.000000,1.000000,1.000000,4319.000000,11342.000000,4570.000000,3484.000000,3021.000000


In [33]:
columnas = ['E-commerce','Económico','Marítimo baleares',\
            'Urgente 10','Urgente 12','Urgente 19','Urgente 19 expedición',\
            'Monday','Tuesday','Wednesday','Thursday','Friday']

In [34]:
#train = train[columnas]
#train.head()

In [35]:
train = train.iloc[:,1:-5]

In [36]:
train.head()

,Bag 14,Bag 19,Burofax,Devolución burofax,Documentos 14,E-Commerce,E-commerce,E-commerce canje,Ecobox 25,Económico,Express documentos,Express paquetes,Marítimo baleares,Mascotas,Mrw economy (ecopaq),OperativaServicioEnumECOBA,Plan ayuda,Retorno albarán,Retorno burofax,Retorno canje,Urgente 10,Urgente 10 expedición,Urgente 12,Urgente 12 expedición,Urgente 14,Urgente 14 expedición,Urgente 19,Urgente 19 expedición,Urgente 8:30,Urgente 8:30 expedición,Urgente funchal,Urgente hoy
Fecha envío,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,21.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0
2020-01-06,0.0,1.0,1.0,0.0,0.0,0.0,161.0,0.0,0.0,35.0,0.0,0.0,152.0,6.0,0.0,0.0,0.0,1.0,0.0,0.0,69.0,1.0,164.0,16.0,0.0,0.0,497.0,22.0,0.0,0.0,0.0,0.0
2020-01-07,0.0,0.0,0.0,0.0,0.0,0.0,396.0,0.0,0.0,142.0,0.0,0.0,186.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
2020-01-08,0.0,0.0,6.0,0.0,0.0,0.0,23.0,0.0,0.0,4.0,0.0,0.0,595.0,23.0,0.0,0.0,0.0,1.0,0.0,0.0,127.0,6.0,172.0,12.0,1.0,0.0,100.0,33.0,1.0,1.0,0.0,0.0
2020-01-09,0.0,0.0,8.0,0.0,0.0,0.0,1381.0,0.0,0.0,382.0,0.0,0.0,97.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,129.0,6.0,176.0,14.0,0.0,0.0,144.0,60.0,2.0,1.0,0.0,0.0


In [37]:
'''
train.Monday = [ 1 if monday > 0 else 0 for monday in train.Monday]
train.Tuesday = [ 1 if monday > 0 else 0 for monday in train.Tuesday]
train.Wednesday = [ 1 if monday > 0 else 0 for monday in train.Wednesday]
train.Thursday = [ 1 if monday > 0 else 0 for monday in train.Thursday]
train.Friday = [ 1 if monday > 0 else 0 for monday in train.Friday]
train.head()
'''

'\ntrain.Monday = [ 1 if monday > 0 else 0 for monday in train.Monday]\ntrain.Tuesday = [ 1 if monday > 0 else 0 for monday in train.Tuesday]\ntrain.Wednesday = [ 1 if monday > 0 else 0 for monday in train.Wednesday]\ntrain.Thursday = [ 1 if monday > 0 else 0 for monday in train.Thursday]\ntrain.Friday = [ 1 if monday > 0 else 0 for monday in train.Friday]\ntrain.head()\n'

In [38]:
train.to_pickle('./data/df_Envios')